In [1]:
from glob import glob
from src.scraping import soup_parser
from bs4 import BeautifulSoup
import pandas as pd

In [8]:
pd.set_option('display.max_columns', None)

In [ ]:
# data_list = []
# for f in glob('../html/tw*.html'):
#     with open(f, 'r') as f2:
#         html = f2.read()
#         to_be_parsed = BeautifulSoup(html)
#         f2.close()
#         data = soup_parser(to_be_parsed)
#         data_list.append(data)
# data_list = [x for x in data_list if x != None]
# df = pd.DataFrame(data_list)

In [2]:
df = pd.read_csv('parsed_soup_raw.csv').drop(columns = 'Unnamed: 0')

In [3]:
streamer_df = pd.read_csv('merged_df.csv')
streamer_df.drop(columns = 'Unnamed: 0', inplace = True)

In [4]:
df.login = df.login.map(lambda x: x.split('\n')[1] if '\n' in x else x)
df.login = df.login.str.lower()

In [16]:
working_df = streamer_df.join(df.set_index('login'),
                              on = 'login',
                              lsuffix = '_left',
                              rsuffix = '_right').drop(columns = 'type_right').dropna(axis = 0, subset = {'Hours streamed'})

In [18]:
cleaning_df = working_df.drop(columns = ['user_id',
                                         'user_name',
                                         'viewer_count',
                                         'type_left',
                                         'display_name',
                                         'description',
                                         'created_at',
                                         'profile_image_url',
                                         'offline_image_url',
                                         'game_id',
                                         'Followers',
                                         'Views', 
                                         'Hours watched daily',
                                         'Views to date'
                          ]
               )
cleaning_df.reset_index(drop=True, inplace = True)
cleaning_df.columns = cleaning_df.columns.str.lower().str.replace(' ', '_')
cleaning_df.broadcaster_type = cleaning_df.broadcaster_type.fillna('unpartnered')
cleaning_df.account_age = pd.to_timedelta(cleaning_df.account_age)
cleaning_df.view_count = cleaning_df.view_count.astype('int')
cleaning_list = ['hours_streamed',
             'average_viewers',
             'peak_viewers', 
             'days_of_activity',
             'total_games_streamed',
             'followers_per_stream',
             'views_per_stream',
             'followers_per_hour',
             'views_per_hour',
             'hours_watched',
             'followers_to_date',
             'hours_watched_daily']
cleaning_df[cleaning_list] = cleaning_df[cleaning_list].astype('int')
cleaning_df.average_games = cleaning_df.average_games.map(lambda x: x[:-7]).astype('float')
cleaning_df.active_days_per_week = cleaning_df.active_days_per_week.map(lambda x: x[:-4]).astype('float')
cleaning_df.daily_broadcast_time = cleaning_df.daily_broadcast_time.map(lambda x: x[:-3]).astype('int')
cleaning_df['is_affiliate'] = [0 if x == 'unpartnered' else 1 for x in cleaning_df.broadcaster_type]
cleaning_df['target'] = [1 if x == 'partner' else 0 for x in cleaning_df.broadcaster_type]

In [22]:
cleaning_df.to_csv('streamer_data.csv', index = False)